# Query that returns both text(name) and code(ID) from JSON-stat
## Example HS-codes in foreign trade

### Import libraries
Use [pyjstat](https://pypi.org/project/pyjstat/) library for JSON-stat and pandas. Pandas is loaded as part of av pyjstat

In [1]:
from pyjstat import pyjstat
import requests

### URL with the table's metadata, where we post the query

In [2]:
tabid = "08799" # 
lang = "en" # language code can also be "no"
POST_URL = "https://data.ssb.no/api/v0/" + lang + "/table/" + tabid # 'https://data.ssb.no/api/v0/en/table/08799'

### Query, can be taken from the console: 
Import / export all commodity codes (HS) to US May 2020, ca. 65000 cells. Max limit for one query in PxWebApi is 800.000 cells, incl. empty cells.

In [3]:
json_q = {
  "query": [
    {
      "code": "Varekoder",
      "selection": {
        "filter": "all",
        "values": [
          "*"
        ]
      }
    },
    {
      "code": "ImpEks",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    },
    {
      "code": "Land",
      "selection": {
        "filter": "item",
        "values": [
          "US"
        ]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "Mengde1",
          "Verdi",
          "Mengde2"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2020M05"
        ]
      }
    }
  ],
  "response": {
    "format": "json-stat2"
  }
}


### Post query

In [4]:
res = requests.post(POST_URL, json=json_q)

### Read JSON-stat result using the library pyjstat
Saved it as dataset ds.

In [5]:
ds = pyjstat.Dataset.read(res.text)

In [6]:
type(ds)

pyjstat.pyjstat.Dataset

Check dataset ds

In [7]:
ds

Dataset([('class', 'dataset'),
         ('label',
          '08799: External trade in goods, by commodity number, imports/exports, country, contents and month'),
         ('source', 'Statistics Norway'),
         ('updated', '2020-06-15T06:00:00Z'),
         ('id', ['Varekoder', 'ImpEks', 'Land', 'ContentsCode', 'Tid']),
         ('size', [10767, 2, 1, 3, 1]),
         ('dimension',
          OrderedDict([('Varekoder',
                        OrderedDict([('label', 'commodity number'),
                                     ('category',
                                      OrderedDict([('index',
                                                    OrderedDict([('00000011',
                                                                  0),
                                                                 ('00000031',
                                                                  1),
                                                                 ('00000032',
                        

### Get some main metadata from the JSON-stat dataset

In [8]:
title = ds['label']
print(title)

08799: External trade in goods, by commodity number, imports/exports, country, contents and month


Last update as GMT

In [9]:
last_update = ds['updated']
print(last_update)

2020-06-15T06:00:00Z


Get source

In [10]:
source = ds['source']
print(source)

Statistics Norway


Role gives some shortcuts

In [11]:
ds_roles = ds['role']
print(ds_roles)

OrderedDict([('time', ['Tid']), ('metric', ['ContentsCode']), ('geo', ['Land'])])


### Make dataframes
We have to make two dataframes, one with text and one with id. Pyjstat returns 'label' by default.

In [12]:
hstrade = ds.write('dataframe')
hstrade_id = ds.write('dataframe', naming='id')

In [13]:
hstrade.head()

,commodity number,imports/exports,country,contents,month,value
0,"(q1=kg, q2=none) Imports of unspecified goods,...",Imports,United States,Quantity 1 (Q1),2020M05,0
1,"(q1=kg, q2=none) Imports of unspecified goods,...",Imports,United States,Value (NOK),2020M05,0
2,"(q1=kg, q2=none) Imports of unspecified goods,...",Imports,United States,Quantity 2 (Q2),2020M05,0
3,"(q1=kg, q2=none) Imports of unspecified goods,...",Exports,United States,Quantity 1 (Q1),2020M05,0
4,"(q1=kg, q2=none) Imports of unspecified goods,...",Exports,United States,Value (NOK),2020M05,0


In [14]:
hstrade_id.head()

,Varekoder,ImpEks,Land,ContentsCode,Tid,value
0,00000011,1,US,Mengde1,2020M05,0
1,00000011,1,US,Verdi,2020M05,0
2,00000011,1,US,Mengde2,2020M05,0
3,00000011,2,US,Mengde1,2020M05,0
4,00000011,2,US,Verdi,2020M05,0


### Make new column with ID and label concatinated

In [15]:
hstrade['hstrade_combi'] = hstrade_id['Varekoder'] + ' ' + hstrade['commodity number']

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


In [16]:
hstrade.columns

Index(['commodity number', 'imports/exports', 'country', 'contents', 'month',
       'value', 'hstrade_combi'],
      dtype='object')

### Make new dataframe with only the colums we want in new order. Mark double [[ ]]

In [17]:
hstrade_new = hstrade[['hstrade_combi', 'imports/exports', 'country', 'contents', 'month',
       'value']]

In [18]:
hstrade_new.tail()

,hstrade_combi,imports/exports,country,contents,month,value
64597,"99999999 (q1=kg, q2=none) Total, all commoditi...",Imports,United States,Value (NOK),2020M05,39228737
64598,"99999999 (q1=kg, q2=none) Total, all commoditi...",Imports,United States,Quantity 2 (Q2),2020M05,0
64599,"99999999 (q1=kg, q2=none) Total, all commoditi...",Exports,United States,Quantity 1 (Q1),2020M05,46531687
64600,"99999999 (q1=kg, q2=none) Total, all commoditi...",Exports,United States,Value (NOK),2020M05,137474174
64601,"99999999 (q1=kg, q2=none) Total, all commoditi...",Exports,United States,Quantity 2 (Q2),2020M05,0
